In [ ]:
### imported by jim 4/26/20
### super slow, but who cares it only has to run once
#streamlined into functional structure 5/02/20 jchd
#forked and modified into noise analysis ~5/03/20
from scipy.special import comb
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # this is why I'm using kaggle, pyplot isn't installing on yuffa machine (rabbit)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from random import random
import seaborn as sns
import os
import gc
#!pip install joypy
#import joypy

train_data=pd.read_csv('../input/data-without-drift/train_clean.csv')
test_data=pd.read_csv('../input/data-without-drift/test_clean.csv')
wavebase=pd.read_csv('../input/new-baseline-all-and-synth-2mil-lgbmoofs/submission_wavenet.csv')
test_data['open_channels']=wavebase.open_channels
#create dictionaries for train,test,wavenet dataframes
wavebase_dict={}
for i in range(0,20):
    wavebase_dict[i]=wavebase[100000*i:(i+1)*100000]
    wavebase_dict[i].reset_index(inplace=True,drop=True)
test_dict={}
for i in range(0,20):
    test_dict[i]=test_data[100000*i:(i+1)*100000]
    test_dict[i].reset_index(inplace=True,drop=True)
train_dict={}
for i in range(0,10):
    train_dict[i]=train_data[i*500000:(i+1)*500000]
    train_dict[i].reset_index(inplace=True,drop=True)

    
#let's look at our oofs on train and test
train_oof_data=pd.read_csv('../input/oofs-g94/oofset_train.csv')
test_oof_data=pd.read_csv('../input/oofs-g94/oofs_test.csv')
test_oof_dict={}
for i in range(0,20):
    test_oof_dict[i]=test_oof_data[100000*i:(i+1)*100000]
    test_oof_dict[i].reset_index(inplace=True,drop=True)
    test_oof_dict[i]['signal']=test_dict[i]['signal']
train_oof_dict={}
for i in range(0,10):
    train_oof_dict[i]=train_oof_data[i*500000:(i+1)*500000]
    train_oof_dict[i].reset_index(inplace=True,drop=True)
    train_oof_dict[i]['signal']=train_dict[i]['signal']
#simple check on oofs, should see high correlation between oofs and train open channels
for i in range(0,10):
    print(train_oof_dict[i].oofs.corr(train_dict[i].open_channels))
    print(train_oof_dict[i].signal.corr(train_dict[i].signal))
    print(test_oof_dict[i].signal.corr(test_dict[i].signal))

#create signalp and noisep columns for train dataframes
for group in range(0,10):
    train_dict[group]['signalp']=train_dict[group].open_channels #placeholder, gets overwritten
    for i in range(0,11):
        train_dict[group].loc[train_dict[group].open_channels==i,'signalp']=train_dict[group][train_dict[group].open_channels==i].signal.mean()
    train_dict[group]['noisep']=train_dict[group].signal-train_dict[group].signalp

for group in range(0,20):
    test_dict[group]['signalp']=test_dict[group].open_channels #placeholder, gets overwritten

    for i in range(0,11):
        test_dict[group].loc[test_dict[group].open_channels==i,'signalp']=test_dict[group][test_dict[group].open_channels==i].signal.mean()
    test_dict[group]['noisep']=test_dict[group].signal-test_dict[group].signalp

for group in range(0,20):
    test_oof_dict[group]['signalp']=test_oof_dict[group].open_channels #placeholder, gets overwritten
    for i in range(0,11):
        test_oof_dict[group].loc[test_oof_dict[group].open_channels==i,'signalp']=test_oof_dict[group][test_oof_dict[group].open_channels==i].signal.mean()
    test_oof_dict[group]['noisep']=test_oof_dict[group].signal-test_oof_dict[group].signalp


class model1():
    def __init__(self,rates, size=10):
        self.size=size
        self.rates=rates
        self.channels=[]
        for i in range(0,self.size):
            self.channels.append('a')
    def reset(self):
        self.channels=[]
        for i in range(0,self.size):
            self.channels.append('a')
    def value(self):
        return self.channels.count('a')+self.channels.count('b')+self.channels.count('c')
    def get_state(self):
        x=[]
        for i in ['a','b','c','d','e','f']:
            x.append(self.channels.count(i))
        return(x)
    def wiggle(self):
        #print('old state:',self.channels)
        #print(self.value())
        choices=['a','b','c','d','e','f']
        psum=9813.5
        for i in range(0,len(self.channels)):
            if self.channels[i]=='a':
                
                np.rand(1)
        for i in range(0,len(self.channels)):
            if self.channels[i]=='a':
                newstate=np.random.choice(choices,1,p=self.rates[0])[0]
            if self.channels[i]=='b':
                newstate=np.random.choice(choices,1,p=self.rates[1])[0]
            if self.channels[i]=='c':
                newstate=np.random.choice(choices,1,p=self.rates[2])[0]
            if self.channels[i]=='d':
                newstate=np.random.choice(choices,1,p=self.rates[3])[0]
            if self.channels[i]=='e':
                newstate=np.random.choice(choices,1,p=self.rates[4])[0]
            if self.channels[i]=='f':
                newstate=np.random.choice(choices,1,p=self.rates[5])[0] 
            self.channels[i]=newstate
        #print('new state:',self.channels)
        #print(self.value())
    def run(self,n):
        x=[]
        for i in range(0,n):
            x.append(self.value())
            self.wiggle()
        return(x)
    def state_run(self,n):
        x=[]
        for i in range(0,n):
            x.append(self.get_state())
            self.wiggle()
        y=pd.DataFrame(x,columns=['a','b','c','d','e','f'])
        y['open_channels']=y.a+y.b+y.c

        return(y)

rates1=[[0.997, 3.060e-3, 2.946e-5, 3.618e-6, 1.020e-8, 5.018e-12],
       [1.283e-3, 0.978, 1.788e-2, 3.279e-3, 1.264e-5, 7.872e-9],
       [2.267e-5, 0.033, 0.699, 0.266, 1.637e-3, 1.393e-6],
       [2.763e-6, 5.972e-3, 0.264, 0.721, 9.047e-3, 1.144e-5],
       [1.872e-8, 5.532e-5, 3.904e-3, 2.175e-2, 0.972, 2.344e-3],
       [7.291e-11, 2.728e-7, 2.630e-5, 2.176e-4, 1.855e-2, 0.981]]
#fix this bullshit normalization below and just normalize the columns!!
for i in range(0,len(rates1)):
    rowsum=0
    for j in range(0,len(rates1[i])):
        if i!=j:
            rates1[i][j]=round(rates1[i][j],5)
            rowsum=rowsum+rates1[i][j]  
    rates1[i][i]=1-rowsum

#rates2 seem superior
rates2=[[0.997, 3.159e-3, 3.042e-5, 3.728e-6, 1.052e-8, 4.957e-12],
  [1.185e-3, 0.978, 1.789e-2, 3.272e-3, 1.262e-5, 7.532e-9],
  [2.093e-5, 0.033, 0.7, 0.265, 1.633e-3, 1.332e-6],
  [2.598e-6, 6.080e-3, 0.269, 0.716, 9.018e-3, 1.093e-5],
  [1.761e-8, 5.635e-5, 3.974e-3, 2.168e-2, 0.972, 2.245e-3],
  [7.220e-11, 2.925e-7, 2.820e-5, 2.285e-4, 1.952e-2, 0.98]]

for i in range(0,len(rates2)):
    tot=0
    for j in range(0,len(rates2[i])):
        tot=tot+rates2[i][j]
    for j in range(0,len(rates2[i])):
        rates2[i][j]=rates2[i][j]/tot


#fix this bullshit normalization below and just normalize the columns!!
#for i in range(0,len(rates2)):
#    rowsum=0
#    for j in range(0,len(rates2[i])):
#        if i!=j:
#            rates2[i][j]=round(rates2[i][j],5)
#            rowsum=rowsum+rates2[i][j]  
#    rates2[i][i]=1-rowsum

#adds/overwrites pure signal column for synthetic in column name as argument, averages traingroup arguments for values
def generate_pure_signal(synthx,name,traingroup1,traingroup2):
    print('creating signal column for synth based on signalp in group',str(traingroup1),'and',str(traingroup2))
    train_data1=train_dict[traingroup1]#[0:100000]
    train_data2=train_dict[traingroup2]#[0:100000]
    synthx[name]=synthx.open_channels #placeholder, gets overwritten
    for i in range(0,11):
        synthx.loc[synthx.open_channels==i,name]=(train_data1[train_data1.open_channels==i].signal.mean()+train_data2[train_data2.open_channels==i].signal.mean())/2.
    synthx.reset_index(drop=True,inplace=True)
    gc.collect()

def add_noise_make_signal(synth,name,traingroup,reverse=False):
    print('adding noise from',str(traingroup))
    synth[name]=synth.signalp+train_dict[traingroup].noisep #placeholder, should overwrite
    if reverse:
            synth[name]=synth.signalp+train_dict[traingroup].noisep.values[::-1] #placeholder, should overwrite

def check_synth_signal(synth,traingroup):
    plt.plot(synth.signal[0:100000])
    plt.plot(train_dict[traingroup].signal[0:100000])
    plt.show()
    plt.hist(synth.open_channels,bins=30)
    plt.hist(train_dict[traingroup].open_channels,bins=80)
    plt.show()
    


In [ ]:
def generatetrainautocorr(train_data,max=1000):
    train_autocorr_dict={}
    for group in range(0,10):
        orig = train_dict[group][0:100000]
        origautocorr=[]
        for i in range(1,1000):
            origautocorr.append(orig.open_channels.diff(i).value_counts()[0])
        print('generated autocorr to max',str(max),'for train group',group)
        train_autocorr_dict[group]=origautocorr
    return(train_autocorr_dict)
        
def checkrun(synth,group=False,train=False):
    #length=len(synth)
    if group:
        print('comparing synth(blue) to train_autocorr group',str(group),'(orange)')
        orig =train_dict[group][0:100000]
        orig.reset_index(inplace=True,drop=True)
        orig.reset_index(inplace=True,drop=True)
        origautocorr=train_autocorr_dict[group]
    #for i in range(0,len(origautocorr)):
    #    origautocorr[i]=origautocorr[i]/length
    if not group:
        orig =train
        print('no group given, comparing synth(blue) to input set(orange)')
        origautocorr=[]
        for i in range(1,1000):
            origautocorr.append(train.open_channels.diff(i).value_counts()[0])
        print('generated trainautocorr')
    synthautocorr=[]
    for i in range(1,1000):
        synthautocorr.append(synth.open_channels.diff(i).value_counts()[0])
    print('generated synthautocorr')
    synthautocorr=[]
    for i in range(1,1000):
        synthautocorr.append(synth.open_channels.diff(i).value_counts()[0])
    print('generated synthautocorr')
    #for i in range(0,len(synthautocorr)):
    #    synthautocorr[i]=synthautocorr[i]/length
    
    plt.plot(synthautocorr,linewidth=3)#[0:500000])
    plt.plot(origautocorr)
    plt.xlim(-10, 10)
    plt.show()
    plt.plot(synthautocorr,linewidth=3)#[0:500000])
    plt.plot(origautocorr)
    plt.xlim(-10, 70)
    plt.show()
    plt.plot(synthautocorr,linewidth=3)#[0:500000])
    plt.plot(origautocorr)
    plt.xlim(-10, 200)
    plt.show()
    plt.plot(synthautocorr,linewidth=3)#[0:500000])
    plt.plot(origautocorr)
    plt.xlim(-10, 1000)
    plt.show()
    plt.xlim(-1, 16)
    plt.hist(synth.open_channels,range=[0,50],bins=50)
    plt.hist(orig.open_channels,bins=30)
    plt.show()
    
    
train_autocorr_dict=generatetrainautocorr(train_data)

In [ ]:
def create_synthetic(length=500000,size=10,rates=rates2,puregroups=[4,9],noisegroup=9,reverse=False,usetestnoise=False):
    mm=model1(rates,size=size)
    synth=mm.state_run(length)
    checkrun(synth,group=puregroups[0])
    checkrun(synth,group=puregroups[1])
    generate_pure_signal(synth,'signalp',puregroups[0],puregroups[1])
    add_noise_make_signal(synth,'signal',noisegroup,reverse=reverse)
    check_synth_signal(synth,puregroups[0])
    check_synth_signal(synth,puregroups[1])
    print(synth.head())
    return synth

def create_synthetic_hack(length=100000,size=10,rates=rates2,puregroups=[4,9],noisegroup=9,reverse=False,usetestnoise=False):
    print('HACK VERSION ONLY CREATES runs of 100000')
    mm=model1(rates,size=size)
    synth=mm.state_run(length)
    #checkrun(synth,group=puregroups[0])
    #checkrun(synth,group=puregroups[1])
    print('NOT Making autocorrelation check plots! (DONT worrry duude, rates are fine)')
    generate_pure_signal_hack(synth,'signalp',puregroups[0],puregroups[1])
    add_noise_make_signal_hack(synth,'signal',noisegroup,reverse=reverse)
    #check_synth_signal(synth,puregroups[0])
    #check_synth_signal(synth,puregroups[1])
    print('NOT running check_synth_signal, - not informative')
    check_synth_vs_test5_hack(synth)
    compare_synth_test5_50Hz(synth)
    print(synth.head())
    return synth

def generate_pure_signal_hack(synthx,name,traingroup1,traingroup2):
    print('creating signalp column for synth based the list thats hardcoded right here!')
    signalp_list=(-5.5,-4.4,
     -3.1005879806196607,
     -1.6904845933226527,
     -0.5866951569088326,
     0.6522344081285519,
     1.8921235881801317,
     3.1403901473484637,
     4.378914822039075,
     5.601466304727297,
     6.870557241519672)
    synthx[name]=synthx.open_channels #placeholder, gets overwritten
    for i in range(0,11):
        synthx.loc[synthx.open_channels==i,name]=signalp_list[i]
    synthx.reset_index(drop=True,inplace=True)
    gc.collect()

def add_noise_make_signal_hack(synth,name,traingroup,reverse=False):
    print('adding noise based on sin fits to 50Hz and gaussian noise!')
    length=synth.shape[0]
    noise_std=0.4119683951753552
    gaussian_noise=np.random.normal(0,noise_std*.983,size=length)
    print('original std on noise (train):',noise_std)
    print('gaussian component of synthetic noise',gaussian_noise.std())
    x_50Hz=np.arange(0,length)
    A=np.linspace(0.04742,0.1475,length)
    signal_50Hz=0.942*A*np.sin(np.pi/(100)*0.997807*x_50Hz+1.3-5/100)
    synth['noisep']=gaussian_noise+signal_50Hz
    synth[name]=synth.signalp+gaussian_noise+signal_50Hz
    print('total synth noisep std:',(gaussian_noise+signal_50Hz).std())

    synth[name]=synth.signalp+train_dict[traingroup].noisep #placeholder, should overwrite
    if reverse:
            synth[name]=synth.signalp+train_dict[traingroup].noisep.values[::-1] #placeholder, should overwrite

def check_synth_vs_test5_hack(synth):
    fig = plt.figure(figsize = (18, 6))
    ax = fig.add_subplot()

    bins1=150
    n_train, bins_train, patches_train = ax.hist(synth.signal, bins = bins1, color='blue',stacked = False, density = True)
    n_test, bins_test, patches_test = ax.hist(test_dict[5].signal, bins = bins_train, color='red',stacked = False, density = True,alpha=0.6)
    print(test_dict[5].shape)
    print(synth.shape)

    ax.set_xlabel("Signal") 
    ax.set_ylabel("Density")
    ax.set_title('set 5 test (red) and synth (blue) max channel 10 sets')
    handles, labels = ax.get_legend_handles_labels()
    plt.show()
    
def compare_synth_test5_50Hz(synth):
    #shows 50Hz signal in test data
    length=synth.shape[0]
    range1=12800
    fig = plt.figure(figsize = (18, 6))
    ax = fig.add_subplot()
    for j in [0]:
        period=200
        synth['phase']=synth.index+j
        synth['phase']=synth.phase.mod(period)
        averagenoise=[]
        for i in range(0,int(100000/range1)):
            averagenoise=[]
            sett=synth[int(i*range1):int((i+1)*range1)]
            for p in range(0,period):
                averagenoise.append(sett[sett.phase==p].noisep.mean())
            ax.plot(np.arange(i*period,(i+1)*period),averagenoise)
    period=200
    x=np.arange(0,100000)
    A=np.linspace(0.04742,0.1475,100000)
    y=0.942*A*np.sin(np.pi/(100)*0.997807*x+1.3-5/100)
    sinsynth=pd.DataFrame({'x':x,'y':y})
    sinsynth['phase']=sinsynth.x.mod(period)
    averagenoise=[]

    for i in range(0,int(100000/range1)):
        averagenoise=[]
        sett=sinsynth[int(i*range1):int((i+1)*range1)]
        for p in range(0,period):
            averagenoise.append(sett[sett.phase==p].y.mean())
        ax.plot(np.arange(i*period,(i+1)*period),averagenoise)
    ax.set_xlabel("Signal") 
    ax.set_ylabel("Time (averaged, full run)")
    ax.set_title('synthetic data 50Hz sin component')
    
    range1=12800
    fig = plt.figure(figsize = (18, 6))
    ax = fig.add_subplot()
    for j in [0]:
        group=5
        period=200
        test_dict[group]['phase']=test_dict[group].index+j
        test_dict[group]['phase']=test_dict[group].phase.mod(period)
        averagenoise=[]
        for i in range(0,int(100000/range1)):
            averagenoise=[]
            sett=test_dict[group][int(i*range1):int((i+1)*range1)]
            for p in range(0,period):
                averagenoise.append(sett[sett.phase==p].noisep.mean())
            ax.plot(np.arange(i*period,(i+1)*period),averagenoise)


#create a signal (no noise), plot by same method as above, see if plot looks the same (but without noise)
    period=200
    x=np.arange(0,100000)
    A=np.linspace(0.04742,0.1475,100000)
    y=0.942*A*np.sin(np.pi/(100)*0.997807*x+1.3-5/100)
    sinsynth=pd.DataFrame({'x':x,'y':y})
    sinsynth['phase']=sinsynth.x.mod(period)
    #shows 50Hz signal in test data
    averagenoise=[]

    for i in range(0,int(100000/range1)):
        averagenoise=[]
        sett=sinsynth[int(i*range1):int((i+1)*range1)]
        for p in range(0,period):
            averagenoise.append(sett[sett.phase==p].y.mean())
        ax.plot(np.arange(i*period,(i+1)*period),averagenoise)
        ax.set_title('Test set 5 50Hz sin component'+str(j))



In [ ]:
real=pd.read_csv('../input/ion-real-synth1/astr1.csv',header=None)
real['open_channels']=real[0]

In [ ]:
checkrun(real,group=5,train=5)

In [ ]:
rates4=[[10932, 31,0 , 0, 0,   0],#(with T) low then matches ave ok(?)
        [11.5, 10932, 218, 0, 0,   0],
        [0, 353, 10932, 3368, 0, 0],
        [0, 0, 4799, 10932, 109, 0],
        [0, 0, 0, 262, 10932,  24],
        [0, 0, 0, 0, 638,   10932]]
31+11.5+218+353+3368+4799+109+262+24+638

In [ ]:
#generate train autocorr has been modifed to only work on 100000 point sets
#check run also modifed to only work on 100000 point sets

rates3=[[0.997, 3.159e-3, 3.042e-5, 3.728e-6, 1.052e-8, 4.957e-12],
  [1.185e-3, 0.978, 1.789e-2, 3.272e-3, 1.262e-5, 7.532e-9],
  [2.093e-5, 0.033, 0.7, 0.265, 1.633e-3, 1.332e-6],
  [2.598e-6, 6.080e-3, 0.269, 0.716, 9.018e-3, 1.093e-5],
  [1.761e-8, 5.635e-5, 3.974e-3, 2.168e-2, 0.972, 2.245e-3],
  [7.220e-11, 2.925e-7, 2.820e-5, 2.285e-4, 1.952e-2, 0.98]]
ratest=[[0.997, 3.159e-3, 3.042e-5, 3.728e-6, 1.052e-8, 4.957e-12],
  [1.185e-3, 0.978, 1.789e-2, 3.272e-3, 1.262e-5, 7.532e-9],
  [2.093e-5, 0.033, 0.7, 0.265, 1.633e-3, 1.332e-6],
  [2.598e-6, 6.080e-3, 0.269, 0.716, 9.018e-3, 1.093e-5],
  [1.761e-8, 5.635e-5, 3.974e-3, 2.168e-2, 0.972, 2.245e-3],
  [7.220e-11, 2.925e-7, 2.820e-5, 2.285e-4, 1.952e-2, 0.98]]
mid=4000

ratesz=[[10000-(11.5), 11.5,0 , 0, 0,   0],#similar to below (with T)
        [31, 10000-(353+31), 353, 0, 0,   0],
        [0, 218, 10000-(218+4799), 4799, 0, 0],
        [0, 0, 3368, 10000-(3368+262), 262, 0],
        [0, 0, 0, 109, 10000-(109+638),  638],
        [0, 0, 0, 0, 24,   10000-(24)]]
ratesas=[[10000, 11.5,0 , 0, 0,   0],#low (not horrible), then a tiny bit high. ave bit too high
        [31, 10000, 353, 0, 0,   0],#low diff one on autocorr (with T)
        [0, 218, 10000, 4799, 0, 0],
        [0, 0, 3368, 10000, 262, 0],
        [0, 0, 0, 109, 10000,  638],
        [0, 0, 0, 0, 24,   10000]]
rates4=[[9813.5, 11.5,0 , 0, 0,   0],#(with T) low then matches ave ok(?)
        [31, 9813.5, 353, 0, 0,   0],
        [0, 218, 9813.5, 4799, 0, 0],
        [0, 0, 3368, 9813.5, 262, 0],
        [0, 0, 0, 109, 9813.5,  638],
        [0, 0, 0, 0, 24,   9813.5]]
ratest=[[6313.5, 11.5,0 , 0, 0,   0],#
        [31, 6313.5, 353, 0, 0,   0],
        [0, 218, 6313.5, 4799, 0, 0],
        [0, 0, 3368, 6313.5, 262, 0],
        [0, 0, 0, 109, 6313.5,  638],
        [0, 0, 0, 0, 24,   6313.5]]
ratesx=[[(11.5), 11.5,0 , 0, 0,   0],#way too low everywhere (with T)
        [31, (31+353), 353, 0, 0,   0],
        [0, 218, (218+4799), 4799, 0, 0],
        [0, 0, 3368, (3368+262), 262, 0],
        [0, 0, 0, 109, (109+638),  638],
        [0, 0, 0, 0, 24,   (24)]]
ratest=[[0.997, 3.060e-3, 3.005e-5, 3.075e-6, 8.678e-9, 4.240e-12],#original rates from qub(their model)
  [1.135e-3, 0.978, 1.842e-2, 2.781e-3, 1.074e-5, 6.635e-9],#matches at first, quickly is too high
  [1.805e-5, 2.983e-2, 0.744, 0.225, 1.388e-3, 1.170e-6],#(no T) ave too high
  [2.632e-6, 6.419e-3, 0.32, 0.664, 8.693e-3, 1.096e-5],
  [1.786e-8, 5.957e-5, 4.755e-3, 2.089e-2, 0.972, 2.292e-3],
  [2.319e-10, 9.785e-7, 1.065e-4, 7.004e-4, 0.061, 0.938]]
ratesz=[[0.997, 3.060e-3, 0, 0, 0, 0],#original rates from qub(their model) (with zeros)
  [1.135e-3, 0.978, 1.842e-2, 0, 0, 0],#about same as above, maybe not quite as high
  [0, 2.983e-2, 0.744, 0.225, 0, 0],#(no T) ave too high
  [0, 0, 0.32, 0.664, 8.693e-3, 0],
  [0, 0, 0, 2.089e-2, 0.972, 2.292e-3],
  [0, 0, 0, 0, 0.061, 0.938]]
ratesl=[[0.997, 3.060e-3, 3.005e-5, 3.075e-6, 8.678e-9, 4.240e-12],#decrease d to c
  [1.135e-3, 0.978, 1.842e-2, 2.781e-3, 1.074e-5, 6.635e-9],#pretty good, first slightly high
  [1.805e-5, 2.983e-2, 0.744, 0.225, 1.388e-3, 1.170e-6],#
  [2.632e-6, 6.419e-3, 0.22, 0.664, 8.693e-3, 1.096e-5],
  [1.786e-8, 5.957e-5, 4.755e-3, 2.089e-2, 0.972, 2.292e-3],
  [2.319e-10, 9.785e-7, 1.065e-4, 7.004e-4, 0.061, 0.938]]
ratesl=[[0.997, 3.060e-3, 3.005e-5, 3.075e-6, 8.678e-9, 4.240e-12],#decrease d to c
  [1.135e-3, 0.978, 1.842e-2, 2.781e-3, 1.074e-5, 6.635e-9],#pretty good, first slightly high, then low
  [1.805e-5, 2.983e-2, 0.744, 0.225, 1.388e-3, 1.170e-6],#ave too high
  [2.632e-6, 6.419e-3, 0.32-8.693e-3, 0.664, 8.693e-3, 1.096e-5],
  [1.786e-8, 5.957e-5, 4.755e-3, 2.089e-2, 0.972, 2.292e-3],
  [2.319e-10, 9.785e-7, 1.065e-4, 7.004e-4, 0.061, 0.938]]

rates4=[[10932, 31,0 , 0, 0,   0],#(with T) low then matches ave ok(?)
        [11.5, 10932, 218, 0, 0,   0],
        [0, 353, 10932, 3368, 0, 0],
        [0, 0, 4799, 10932, 109, 0],
        [0, 0, 0, 262, 10932,  24],
        [0, 0, 0, 0, 638,   10932]]
rates4=np.array(rates4).T.tolist()
def normalize_rates(rates):
    for i in range(0,len(rates)):
        tot=0
        for j in range(0,len(rates[i])):
            tot=tot+rates[i][j]
        for j in range(0,len(rates[i])):
            rates[i][j]=rates[i][j]/tot
    return(rates)
rates5=[[0.997, 3.060e-3, 0, 0, 0, 0],
  [1.086e-3, 0.978, 1.842e-2, 0, 0, 0],
  [0, 2.983e-2, 0.744, 0.225, 0, 0],
  [0, 0, 0.32, 0.664, 8.693e-3, 0],
  [0, 0, 0, 2.089e-2, 0.972, 2.292e-3],
  [0, 0, 0, 0, 0.061, 0.938]]
synth1=create_synthetic(length=100000,size=5,puregroups=[5,8],noisegroup=8,reverse=False,rates=normalize_rates(rates4))

In [ ]:
#rates 
rates3=[[0.997, 3.159e-3, 3.042e-5, 3.728e-6, 1.052e-8, 4.957e-12],
  [1.185e-3, 0.978, 1.789e-2, 3.272e-3, 1.262e-5, 7.532e-9],
  [2.093e-5, 0.033, 0.7, 0.265, 1.633e-3, 1.332e-6],
  [2.598e-6, 6.080e-3, 0.269, 0.716, 9.018e-3, 1.093e-5],
  [1.761e-8, 5.635e-5, 3.974e-3, 2.168e-2, 0.972, 2.245e-3],
  [7.220e-11, 2.925e-7, 2.820e-5, 2.285e-4, 1.952e-2, 0.98]]

for i in range(0,len(rates2)):
    tot=0
    for j in range(0,len(rates2[i])):
        tot=tot+rates2[i][j]
    for j in range(0,len(rates2[i])):
        rates2[i][j]=rates2[i][j]/tot
        
synth1=create_synthetic(length=500000,size=10,puregroups=[4,9],noisegroup=4,reverse=False,rates=rates3)


In [ ]:
runlength=100000
synth_5_5=pd.DataFrame()
#for i in range(0,50):
#    print(i)
synth1=create_synthetic_hack(length=runlength,size=10,puregroups=[4,9],noisegroup=4,reverse=False)
    #synth_5_5=pd.concat([synth_5_5,synth1])
    #synth_5_5.reset_index(inplace=True,drop=True)
#synth_5_5.to_csv('synth5M_5_5.csv')

In [ ]:
'''### add 5 and 5 to get extra 10 channel set
synth5p5_1=pd.DataFrame()
synth5p5_2=pd.DataFrame()
synth5p5_1['open_channels']=train_dict[5].open_channels+train_dict[8].open_channels
synth5p5_2['open_channels']=train_dict[5].open_channels+train_dict[8].open_channels
synth5psynth_1=pd.DataFrame()
synth5psynth_2=pd.DataFrame()
synth5psynth_3=pd.DataFrame()
synth5psynth_4=pd.DataFrame()
synth5psynth_1['open_channels']=train_dict[5].open_channels+synth5_1.open_channels
synth5psynth_2['open_channels']=train_dict[8].open_channels+synth5_2.open_channels
synth5psynth_3['open_channels']=train_dict[5].open_channels+synth5_3.open_channels
synth5psynth_4['open_channels']=train_dict[8].open_channels+synth5_4.open_channels
checkrun(synth5p5_1,group=9)
checkrun(synth5p5_2,group=4)
checkrun(synth5psynth_1,group=9)
checkrun(synth5psynth_2,group=4)

generate_pure_signal(synth5p5_1,'signalp',4,9)
generate_pure_signal(synth5p5_2,'signalp',4,9)
generate_pure_signal(synth5psynth_1,'signalp',4,9)
generate_pure_signal(synth5psynth_2,'signalp',4,9)
generate_pure_signal(synth5psynth_3,'signalp',4,9)
generate_pure_signal(synth5psynth_4,'signalp',4,9)

add_noise_make_signal(synth5p5_1,'signal',4,reverse=False)
add_noise_make_signal(synth5p5_2,'signal',9,reverse=False)
add_noise_make_signal(synth5psynth_1,'signal',4,reverse=False)
add_noise_make_signal(synth5psynth_2,'signal',9,reverse=False)
add_noise_make_signal(synth5psynth_3,'signal',4,reverse=False)
add_noise_make_signal(synth5psynth_4,'signal',9,reverse=False)
    
check_synth_signal(synth5p5_1,4)
check_synth_signal(synth5p5_2,4)
check_synth_signal(synth5psynth_1,4)
check_synth_signal(synth5psynth_2,4)
check_synth_signal(synth5psynth_3,4)
check_synth_signal(synth5psynth_4,4)


synth5p5_1[['signal','open_channels']].to_csv('synth5p5_4noise_500k.csv')
synth5p5_2[['signal','open_channels']].to_csv('synth5p5_9noise_500k.csv')
synth5psynth_1[['signal','open_channels']].to_csv('synth5psynth1_4noise_500k.csv')
synth5psynth_2[['signal','open_channels']].to_csv('synth5psynth2_9noise_500k.csv')
synth5psynth_3[['signal','open_channels']].to_csv('synth5psynth3_4noise_500k.csv')
synth5psynth_4[['signal','open_channels']].to_csv('synth5psynth4_9noise_500k.csv')

add_noise_make_signal(synth5p5_1,'signal',4,reverse=True)
add_noise_make_signal(synth5p5_2,'signal',9,reverse=True)
add_noise_make_signal(synth5psynth_1,'signal',4,reverse=True)
add_noise_make_signal(synth5psynth_2,'signal',9,reverse=True)
add_noise_make_signal(synth5psynth_3,'signal',4,reverse=True)
add_noise_make_signal(synth5psynth_4,'signal',9,reverse=True)

check_synth_signal(synth5p5_1,4)
check_synth_signal(synth5p5_2,4)
check_synth_signal(synth5psynth_1,4)
check_synth_signal(synth5psynth_2,4)
check_synth_signal(synth5psynth_3,4)
check_synth_signal(synth5psynth_4,4)

synth5p5_1[['signal','open_channels']].to_csv('synth5p5_4noisereversed_500k.csv')
synth5p5_2[['signal','open_channels']].to_csv('synth5p5_9noisereversed_500k.csv')
synth5psynth_1[['signal','open_channels']].to_csv('synth5psynth1_4noisereversed_500k.csv')
synth5psynth_2[['signal','open_channels']].to_csv('synth5psynth2_9noisereversed_500k.csv')
synth5psynth_3[['signal','open_channels']].to_csv('synth5psynth3_4noisereversed_500k.csv')
synth5psynth_4[['signal','open_channels']].to_csv('synth5psynth4_9noisereversed_500k.csv')

'''

In [ ]:
'''train_like_shit=pd.concat([train_dict[4],train_dict[9],synth5p5_1,synth5psynth_1,synth5psynth_2,synth5psynth_3,synth5psynth_4,synth1,synth2,synth3])
train_like_shit.reset_index(inplace=True,drop=True)
train_like_shit[['signal','open_channels']].to_csv('train49_5M.csv')'''

In [ ]:
#verify that train and test have similar autocorrelation in group 4/9
'''testautocorr=[]
trainautocorr=[]
for i in range(1,1000):
    testautocorr.append(test_dict[5].signal.diff(i).round(1).value_counts()[0])
    trainautocorr.append(train_dict[4][0:100000].signal.diff(i).round(1).value_counts()[0])
print('ok')
fig =  plt.figure(figsize = (20,10))
axes = fig.add_subplot(111)
plt.plot(testautocorr,linewidth=3)
plt.plot(trainautocorr)'''

In [ ]:
#checkrun(wavebase_dict[5],train=train_dict[9][0:100000])

In [ ]:
#checkrun(pd.concat([wavebase_dict[7],wavebase_dict[5]]),train=train_dict[9][0:200000])

In [ ]:
'''for i in range(0,10):
    print(i)
    print(train_dict[i].groupby('open_channels').noisep.std())
    print(train_dict[i].groupby('open_channels').noisep.mean())

print(test_dict[5].groupby('open_channels').noisep.std())
print(test_dict[5].groupby('open_channels').noisep.mean())'''

In [ ]:
test_dict[7].open_channels.corr(train_dict[8].open_channels)

In [ ]:
new_pred7=pd.read_csv('../input/liverpool-10channels-from-5-subtract-signal/predictions_570_580.csv')
new1_pred7=pd.read_csv('../input/liverpool-10channels-from-5/predictions_570_580.csv')

In [ ]:
(test_dict[7].open_channels==new1_pred7.open_channels).value_counts()

In [ ]:
5674/len(test_dict[7])

In [ ]:
for i in range(0,10):

    print(test_dict[i].open_channels.corr(new1_pred7.open_channels))

In [ ]:
for i in range(0,10):
    print(test_dict[i].open_channels.corr(new_pred7.open_channels))

In [ ]:
n=5
r=6
comb(n+r-1,r-1)

In [ ]:
value_counts_dict=dict(train_dict[5].open_channels.value_counts())
max_chan=5
r=3
consistent_state_dict={}
fraction_n_dict={}
total_data=len(train_dict[5])

for n in range(0,6):
    consistent_states=comb(n+r-1,r-1)*comb((max_chan-n)+r-1,r-1)
    consistent_state_dict[n]=consistent_states
    fraction_n_dict[n]=value_counts_dict[n]/total_data
    print('open_channels =',n,'has',consistent_states,'consistent states')
    print('percent of data with',n, 'open channels is',fraction_n_dict[n])
ave_consistent_states=0
for n in range(0,6):
    ave_consistent_states=ave_consistent_states+fraction_n_dict[n]*consistent_state_dict[n]
print()
print('average number of consistent states is',ave_consistent_states)

In [ ]:
46**3

In [ ]:
sum(fraction_n_dict.values())

In [ ]:
x=dict(train_dict[5].open_channels.value_counts())

In [ ]:
x